In [1]:
import pandas as pd
from collections import Counter
import re
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

comp_color = {
    '魔坦刺': '#858561',
    '扶桑法刺': '#9e71ad',
    '扶桑群雄刺': '#968795',
    '扶桑群雄法刺': '#6603fc',
    '扶桑刺': '#6603fc',
    '魔种': '#a35956',
    '扶桑男刺': '#bfae91',
    '扶桑法': '#8000ff',

    '七射': '#14ffa5',
    '攻辅射': '#42d7f5',
    '弟弟射': '#fceb30',
    '攻辅蛋': '#ffac1c',

    '九战': '#ff0000',
    '稷下战': '#ff5e5e',
    '轻战守约': '#ff8e5e',
    '魏战': '#ff0084',

    '蜀国': '#00cc18',

    '法奶': '#0093f5',
    '弟妹法': '#7900fa',
    '坦法': '#ff5900'
    }

comp_to_brief = {
    '扶桑法刺': '扶桑刺',
    '扶桑群雄刺': '扶桑刺',
    '扶桑群雄法刺': '扶桑刺',
    '扶桑男刺': '扶桑刺',
    '魔种(无天赋)' :'魔种',
    '魔种(有天赋)' :'魔种',
    '封神攻辅射': '攻辅射',
    '吴国攻辅射': '攻辅射',
    '稷下攻辅射': '攻辅射',
    '尧天攻辅射': '攻辅射'
}

rank_to_score = [12, 9, 7, 5, 3, 2, 1, 0]

class Config:
    use_brief_comp = True

pd.set_option('display.max_columns', None)

In [2]:
def preprocess_data(df):
    df['分数'] = df['排名'].apply(lambda x: rank_to_score[x - 1])
    if Config.use_brief_comp:
        df['详细阵容'] = df['阵容']
        df['阵容'] = df['详细阵容'].apply(lambda x: comp_to_brief.get(x, x))
    df['回合数'] = df['胜'] + df['负']
    return df

def check_data_integrity(df):
    df = df.dropna()
    groups = df.groupby(['周数', '分组', '局数'])
    for g_name, g_df in groups:
        assert g_df['排名'].tolist() == [1, 2, 3, 4, 5, 6, 7, 8], g_df['排名'].tolist()
        assert g_df['回合数'].is_monotonic_decreasing, g_df['回合数']
        tmp_l = g_df['回合数'].tolist()
        assert tmp_l[0] == tmp_l[1], g_df['回合数']
        assert len(g_df['俱乐部'].unique()) == len(g_df['俱乐部']), len(g_df['俱乐部'].unique())

    # 全局BP
    groups = df.groupby(['周数', '分组', '俱乐部'])
    for g_name, g_df in groups:
        assert len(g_df['ban位'].unique()) == len(g_df['ban位']), (g_name, len(g_df['ban位'].unique()))

    return df

df = pd.read_excel('result.xlsx')
df = preprocess_data(df)
df = check_data_integrity(df)

display(df.head(3))
display(df.tail(3))

,排名,俱乐部,选手,ban位,阵容,质量,输出,胜,负,周数,分组,局数,游戏版本,分数,详细阵容,回合数
0,1,TES,阿虾,稷,魔坦刺,146,161,20,9,3,低分组,1,21-0,12,魔坦刺,29
1,2,RNG.M,俊仔,稷,扶桑刺,80,168,20,9,3,低分组,1,21-0,9,扶桑法刺,29
2,3,VG,婷婷,卫,扶桑刺,71,115,17,11,3,低分组,1,21-0,7,扶桑群雄刺,28


,排名,俱乐部,选手,ban位,阵容,质量,输出,胜,负,周数,分组,局数,游戏版本,分数,详细阵容,回合数
237,6,KSSC,叶凡,安,蜀国,57,90,13,11,4,高分组,5,21-0,2,蜀国,24
238,7,CW,九戒,稷,魔种,49,62,10,9,4,高分组,5,21-0,1,魔种(有天赋),19
239,8,MQ,暴躁黑猪,吴,魔种,33,34,10,8,4,高分组,5,21-0,0,魔种(无天赋),18


In [3]:
def get_comp_statistics(df):
    def get_most_ban(x):
        c = Counter(x['ban位'])
        return [(i, c[i] / len(x)) for i, count in c.most_common()]

    groups = df.groupby('阵容')
    mean_rank = groups['排名'].mean()
    var_rank = groups['排名'].var()
    sum_occur = groups['排名'].count()
    mean_score = groups['分数'].mean()
    top1_cnt = groups['排名'].apply(lambda x: (x == 1).sum())
    top1_rate = top1_cnt / sum_occur
    top3_cnt = groups['排名'].apply(lambda x: (x <= 3).sum())
    top3_rate = top3_cnt / sum_occur
    most_ban = groups.apply(get_most_ban)
    res = pd.concat([mean_score, mean_rank, var_rank, sum_occur, top1_cnt, top1_rate, top3_cnt, top3_rate, most_ban], axis=1)
    res.columns = ('平均分数', '平均排名', '排名方差', '出场次数', '登顶次数', '登顶率', '前三次数', '前三率', '阵容选用')
    res = res.sort_values(['平均分数', '平均排名', '登顶率', '前三率'], ascending=[False, True, False, False])
    res = res.fillna('-')
    return res

def get_ban_statistics(df):
    def get_most_comp(x):
        c = Counter(x['阵容'])
        return [(i, c[i]) for i, count in c.most_common()]
    groups = df.groupby('ban位')
    mean_rank = groups['排名'].mean()
    var_rank = groups['排名'].var()
    mean_score = groups['分数'].mean()
    sum_occur = groups['排名'].count()
    top1_cnt = groups['排名'].apply(lambda x: (x == 1).sum())
    top1_rate = top1_cnt / sum_occur
    top3_cnt = groups['排名'].apply(lambda x: (x <= 3).sum())
    top3_rate = top3_cnt / sum_occur
    most_comp = groups.apply(get_most_comp)
    res = pd.concat([mean_score, mean_rank, var_rank, sum_occur, top1_cnt, top1_rate, top3_cnt, top3_rate, most_comp], axis=1)
    res.columns = ('平均分数', '平均排名', '排名方差', '出场次数', '登顶次数', '登顶率', '前三次数', '前三率', '阵容选用')
    res = res.sort_values(['平均分数', '平均排名', '登顶率', '前三率'], ascending=[False, True, False, False])
    return res


def get_team_statistics(df):
    def get_most_comp(x):
        c = Counter(x['阵容'])
        return [(i, c[i]) for i, count in c.most_common()]
    groups = df.groupby('俱乐部')
    mean_rank = groups['排名'].mean()
    var_rank = groups['排名'].var()
    mean_score = groups['分数'].mean()
    sum_occur = groups['排名'].count()
    top1_cnt = groups['排名'].apply(lambda x: (x == 1).sum())
    top1_rate = top1_cnt / sum_occur
    top3_cnt = groups['排名'].apply(lambda x: (x <= 3).sum())
    top3_rate = top3_cnt / sum_occur
    most_comp = groups.apply(get_most_comp)
    res = pd.concat([mean_score, mean_rank, var_rank, sum_occur, top1_cnt, top1_rate, top3_cnt, top3_rate, most_comp], axis=1)
    res.columns = ('平均分数', '平均排名', '排名方差', '出场次数', '登顶次数', '登顶率', '前三次数', '前三率', '阵容选用')
    res = res.sort_values(['平均分数', '平均排名', '登顶率', '前三率'], ascending=[False, True, False, False])
    return res

def get_player_statistics(df):
    def get_most_comp(x):
        c = Counter(x['阵容'])
        return [(i, c[i]) for i, count in c.most_common()]
    groups = df.groupby('选手')
    mean_rank = groups['排名'].mean()
    var_rank = groups['排名'].var()
    mean_score = groups['分数'].mean()
    sum_occur = groups['排名'].count()
    top1_cnt = groups['排名'].apply(lambda x: (x == 1).sum())
    top1_rate = top1_cnt / sum_occur
    top3_cnt = groups['排名'].apply(lambda x: (x <= 3).sum())
    top3_rate = top3_cnt / sum_occur
    most_comp = groups.apply(get_most_comp)
    res = pd.concat([mean_score, mean_rank, var_rank, sum_occur, top1_cnt, top1_rate, top3_cnt, top3_rate, most_comp], axis=1)
    res.columns = ('平均分数', '平均排名', '排名方差', '出场次数', '登顶次数', '登顶率', '前三次数', '前三率', '阵容选用')
    res = res.sort_values(['平均分数', '平均排名', '登顶率', '前三率'], ascending=[False, True, False, False])
    return res

display(get_comp_statistics(df))
display(get_ban_statistics(df))
display(get_team_statistics(df))
display(get_player_statistics(df))

,平均分数,平均排名,排名方差,出场次数,登顶次数,登顶率,前三次数,前三率,阵容选用
阵容,,,,,,,,,
攻辅蛋,9.000000,2.000000,-,1,0,0.000000,1,1.000000,"[(蜀, 1.0)]"
弟弟射,7.800000,2.840000,3.64,25,8,0.320000,17,0.680000,"[(蜀, 1.0)]"
弟妹法,7.000000,3.000000,-,1,0,0.000000,1,1.000000,"[(蜀, 1.0)]"
轻战守约,7.000000,3.500000,12.5,2,1,0.500000,1,0.500000,"[(安, 0.5), (吴, 0.5)]"
攻辅射,5.080000,4.440000,6.17333,25,4,0.160000,11,0.440000,"[(卫, 0.36), (蜀, 0.32), (魏, 0.28), (尧, 0.04)]"
蜀国,5.071429,4.285714,4.37363,14,1,0.071429,6,0.428571,"[(魏, 0.35714285714285715), (稷, 0.2142857142857..."
扶桑法,5.000000,4.000000,-,1,0,0.000000,0,0.000000,"[(卫, 1.0)]"
魏战,5.000000,4.000000,-,1,0,0.000000,0,0.000000,"[(安, 1.0)]"
扶桑刺,4.931507,4.465753,5.47451,73,8,0.109589,27,0.369863,"[(卫, 0.3698630136986301), (稷, 0.35616438356164..."


,平均分数,平均排名,排名方差,出场次数,登顶次数,登顶率,前三次数,前三率,阵容选用
ban位,,,,,,,,,
蜀,6.583333,3.520833,4.850621,48,11,0.229167,27,0.562500,"[(弟弟射, 25), (攻辅射, 8), (稷下战, 6), (法奶, 4), (扶桑刺,..."
安,4.860465,4.465116,4.873754,43,6,0.139535,16,0.372093,"[(稷下战, 33), (九战, 3), (蜀国, 3), (轻战守约, 1), (魔种, ..."
稷,4.857143,4.547619,5.619628,42,6,0.142857,15,0.357143,"[(扶桑刺, 26), (魔种, 9), (魔坦刺, 4), (蜀国, 3)]"
卫,4.833333,4.437500,4.549202,48,4,0.083333,17,0.354167,"[(扶桑刺, 27), (法奶, 10), (攻辅射, 9), (扶桑法, 1), (蜀国,..."
魏,4.170732,4.926829,5.269512,41,2,0.048780,13,0.317073,"[(扶桑刺, 14), (七射, 9), (攻辅射, 7), (稷下战, 6), (蜀国, 5)]"
吴,2.444444,6.111111,4.861111,9,1,0.111111,1,0.111111,"[(蜀国, 2), (稷下战, 2), (轻战守约, 1), (扶桑刺, 1), (九战, ..."
神,2.000000,6.000000,NaN,1,0,0.000000,0,0.000000,"[(魔坦刺, 1)]"
尧,1.750000,6.500000,3.142857,8,0,0.000000,1,0.125000,"[(七射, 3), (扶桑刺, 2), (攻辅射, 1), (稷下战, 1), (坦法, 1)]"


,平均分数,平均排名,排名方差,出场次数,登顶次数,登顶率,前三次数,前三率,阵容选用
俱乐部,,,,,,,,,
NMG,6.3,3.6,4.266667,10,2,0.2,6,0.6,"[(扶桑刺, 4), (弟弟射, 2), (蜀国, 1), (攻辅射, 1), (稷下战, ..."
CW,6.3,3.9,8.544444,10,3,0.3,5,0.5,"[(扶桑刺, 4), (魔种, 2), (弟弟射, 2), (蜀国, 1), (稷下战, 1)]"
KS.YTG,6.1,3.8,5.511111,10,2,0.2,5,0.5,"[(扶桑刺, 6), (稷下战, 3), (攻辅射, 1)]"
西安WE,6.0,3.7,4.233333,10,1,0.1,4,0.4,"[(扶桑刺, 3), (攻辅射, 2), (稷下战, 1), (七射, 1), (魔种, 1..."
XQ,6.0,3.9,6.766667,10,1,0.1,5,0.5,"[(扶桑刺, 5), (弟弟射, 2), (攻辅射, 1), (稷下战, 1), (蜀国, 1)]"
MQ,5.6,4.3,8.455556,10,3,0.3,4,0.4,"[(扶桑刺, 4), (魔种, 2), (稷下战, 2), (七射, 1), (弟弟射, 1)]"
重庆QGScholar,5.6,4.4,9.822222,10,3,0.3,5,0.5,"[(扶桑刺, 6), (弟弟射, 2), (稷下战, 2)]"
RSG,5.5,4.0,4.000000,10,1,0.1,5,0.5,"[(稷下战, 2), (扶桑刺, 2), (魔坦刺, 1), (攻辅射, 1), (弟弟射,..."
成都AG超玩会,5.5,4.1,5.211111,10,1,0.1,4,0.4,"[(扶桑刺, 4), (弟弟射, 2), (七射, 1), (扶桑法, 1), (蜀国, 1..."


,平均分数,平均排名,排名方差,出场次数,登顶次数,登顶率,前三次数,前三率,阵容选用
选手,,,,,,,,,
简若,12.0,1.000000,0.000000,3,3,1.000000,3,1.0,"[(弟弟射, 2), (扶桑刺, 1)]"
童小话,11.0,1.333333,0.333333,3,2,0.666667,3,1.0,"[(扶桑刺, 3)]"
奕将,10.0,1.666667,0.333333,3,1,0.333333,3,1.0,"[(弟弟射, 2), (稷下战, 1)]"
小浪晨,9.5,2.000000,2.000000,2,1,0.500000,2,1.0,"[(攻辅射, 1), (法奶, 1)]"
伊夜,9.0,2.000000,NaN,1,0,0.000000,1,1.0,"[(弟弟射, 1)]"
...,...,...,...,...,...,...,...,...,...
莫然,1.0,7.000000,NaN,1,0,0.000000,0,0.0,"[(九战, 1)]"
风流,1.0,7.000000,NaN,1,0,0.000000,0,0.0,"[(扶桑刺, 1)]"
流星,0.5,7.500000,0.500000,2,0,0.000000,0,0.0,"[(扶桑刺, 1), (七射, 1)]"


In [4]:
%%time

def report_mvp_comp(comp_stats):
    MIN_OCCURENCE = 5
    metric_weights = {
        '平均分数': [15, False],
        '平均排名': [5, True],
        '排名方差': [3, True],
        '出场次数': [5, False],
        '登顶率': [5, False],
        '前三率': [3, False]
    }
    comp_stats = comp_stats[comp_stats['出场次数'] >= MIN_OCCURENCE]
    raw_ranks = []
    weights = []
    for metric, value in metric_weights.items():
        weight, ascending = value
        metric_rank = comp_stats[metric].rank(ascending=ascending, method='min').astype(int)
        raw_ranks.append(metric_rank)
        weights.append(weight)
    comp_rank = sum([r * w for r, w in zip(raw_ranks, weights)]).sort_values()
    # print(comp_rank)
    mvp_comp = comp_rank.index[0]
    # print('本期MVP阵容:', mvp_comp)
    mvp_comp_metric_ranks = pd.Series({r.name: r[mvp_comp] for r in raw_ranks})
    mvp_comp_metric_values = comp_stats.loc[mvp_comp][mvp_comp_metric_ranks.index].T
    res = pd.concat([mvp_comp_metric_values, mvp_comp_metric_ranks], axis=1)
    res.columns = [mvp_comp, '全阵容对比排名']
    return res
comp_stats = get_comp_statistics(df)
display(report_mvp_comp(comp_stats))

def report_mvp_player(player_stats):
    MIN_OCCURENCE = 3
    metric_weights = {
        '平均分数': [15, False],
        '平均排名': [10, True],
        '排名方差': [5, True],
        '登顶率': [10, False],
        '前三率': [5, False],
        '出场次数': [0, False],
        '阵容选用': [0, False]
    }
    player_stats = player_stats[player_stats['出场次数'] >= MIN_OCCURENCE]
    raw_ranks = []
    weights = []
    for metric, value in metric_weights.items():
        weight, ascending = value
        metric_rank = player_stats[metric].rank(ascending=ascending, method='min').astype(int)
        raw_ranks.append(metric_rank)
        weights.append(weight)
    player_rank = sum([r * w for r, w in zip(raw_ranks, weights)]).sort_values()
    mvp_player = player_rank.index[0]
    # print('本期MVP选手:', mvp_player)
    mvp_player_metric_ranks = pd.Series({r.name: r[mvp_player] for r in raw_ranks})
    mvp_player_metric_values = player_stats.loc[mvp_player][mvp_player_metric_ranks.index].T
    res = pd.concat([mvp_player_metric_values, mvp_player_metric_ranks], axis=1)
    res.columns = [mvp_player, '全选手对比排名']
    res['全选手对比排名'] = (res['全选手对比排名'] / weights * weights).fillna('/')
    return res

player_stats = get_player_statistics(df)
display(report_mvp_player(player_stats))

def report_mvp_team(team_stats):
    metric_weights = {
        '平均分数': [15, False],
        '平均排名': [10, True],
        '排名方差': [5, True],
        '登顶率': [10, False],
        '前三率': [5, False],
        '出场次数': [0, False],
        '阵容选用': [0, False]
    }
    raw_ranks = []
    weights = []
    for metric, value in metric_weights.items():
        weight, ascending = value
        metric_rank = team_stats[metric].rank(ascending=ascending, method='min').astype(int)
        raw_ranks.append(metric_rank)
        weights.append(weight)
    team_rank = sum([r * w for r, w in zip(raw_ranks, weights)]).sort_values()
    mvp_team = team_rank.index[0]
    # print('本期MVP选手:', mvp_team)
    mvp_team_metric_ranks = pd.Series({r.name: r[mvp_team] for r in raw_ranks})
    mvp_team_metric_values = team_stats.loc[mvp_team][mvp_team_metric_ranks.index].T
    res = pd.concat([mvp_team_metric_values, mvp_team_metric_ranks], axis=1)
    res.columns = [mvp_team, '全队伍对比排名']
    res['全队伍对比排名'] = (res['全队伍对比排名'] / weights * weights).fillna('/')
    return res

team_stats = get_team_statistics(df)
display(report_mvp_team(team_stats))

,弟弟射,全阵容对比排名
平均分数,7.8,1
平均排名,2.84,1
排名方差,3.64,1
出场次数,25,3
登顶率,0.32,1
前三率,0.68,1


,简若,全选手对比排名
平均分数,12,1
平均排名,1,1
排名方差,0,1
登顶率,1,1
前三率,1,1
出场次数,3,/
阵容选用,"[(弟弟射, 2), (扶桑刺, 1)]",/


,NMG,全队伍对比排名
平均分数,6.3,1
平均排名,3.6,1
排名方差,4.26667,6
登顶率,0.2,4
前三率,0.6,1
出场次数,10,/
阵容选用,"[(扶桑刺, 4), (弟弟射, 2), (蜀国, 1), (攻辅射, 1), (稷下战, ...",/


Wall time: 127 ms


In [5]:
def report_interesting_data(df):
    b_df = df.copy()
    b_df['选手'] = b_df['选手'].astype(str)
    b_df['俱乐部'] = b_df['俱乐部'].astype(str)
    b_df['选手'] = b_df[['俱乐部', '选手']].agg('.'.join, axis=1)
    b_df['局数'] = b_df.apply(lambda x: f'第{x["周数"]}周{x.loc["分组"]}第{x.loc["局数"]}局', axis=1)
    # 最长游戏回合
    longest_game = b_df.loc[b_df['回合数'] == b_df['回合数'].max()][['选手', '详细阵容', '回合数', '局数', '排名']]
    print(f'最长游戏:\n{longest_game}\n')
    # 最短游戏回合
    first_twos = b_df.loc[b_df['排名'] <= 2]
    shortest_game = first_twos.loc[first_twos['回合数'] == first_twos['回合数'].min()][['选手', '详细阵容', '回合数', '局数', '排名']]
    print(f'最短游戏:\n{shortest_game}\n')
    highest_price = b_df.iloc[b_df['质量'].idxmax()][['选手', '详细阵容', '质量', '局数', '排名']]
    print(f'最高质量:\n{highest_price}\n')
    lowest_price = b_df.iloc[b_df['质量'].idxmin()][['选手', '详细阵容', '质量', '局数', '排名']]
    print(f'最低质量:\n{lowest_price}\n')
    highest_damage = b_df.iloc[b_df['输出'].idxmax()][['选手', '详细阵容', '输出', '局数', '排名']]
    print(f'最高输出:\n{highest_damage}\n')
    lowest_damage = b_df.iloc[b_df['输出'].idxmin()][['选手', '详细阵容', '输出', '局数', '排名']]
    print(f'最低输出:\n{lowest_damage}\n')

report_interesting_data(df)

最长游戏:
              选手    详细阵容  回合数         局数  排名
120  武汉eStar.小玄策  扶桑群雄法刺   36  第4周低分组第1局   1
121     XROCK.痘痘     弟弟射   36  第4周低分组第1局   2
144       RW侠.种子   稷下攻辅射   36  第4周低分组第4局   1
145   杭州LGD大鹅.六月     稷下战   36  第4周低分组第4局   2

最短游戏:
              选手 详细阵容  回合数         局数  排名
128  武汉eStar.曹老大   蜀国   26  第4周低分组第2局   1
129     WB.TS.刺儿  弟弟射   26  第4周低分组第2局   2

最高质量:
选手          CW.简若
详细阵容         扶桑法刺
质量            177
局数      第3周中分组第1局
排名              1
Name: 40, dtype: object

最低质量:
选手        MQ.暴躁黑猪
详细阵容      魔种(无天赋)
质量             33
局数      第4周高分组第5局
排名              8
Name: 239, dtype: object

最高输出:
选手         RW侠.种子
详细阵容        稷下攻辅射
输出            260
局数      第4周低分组第4局
排名              1
Name: 144, dtype: object

最低输出:
选手      KSSC.抓只好棋
详细阵容        扶桑群雄刺
输出              5
局数      第4周高分组第2局
排名              8
Name: 215, dtype: object



In [9]:
def plot_ban_to_comp(ban_stats, filepath=None):
    fig, axs = plt.subplots(1, len(ban_stats), figsize=(30, 4))
    for ax, ban in zip(axs, ban_stats.itertuples()):
        labels, ys = [m[0] for m in ban[-1]], [m[1] for m in ban[-1]]
        colors = [comp_color[re.sub(r'\([^)]*\)', '', c).strip()] for c in labels]
        explode = [0.015] * len(ys)
        def value(val):
            return f'{val:.2f}%'
        ax.pie(ys, labels=labels, colors=colors, explode=explode, autopct=value, shadow=False, pctdistance=0.6)
        ax.set_title(f'{ban[0]} (均分{ban[1]:.2f},出场{ban[4]})', fontsize=15, x=0.5, y=1.01)

    fig.tight_layout()
    if filepath:
        fig.savefig(filepath, dpi=250)
    else:
        plt.show(fig)
    plt.close(fig)

def plot_comp_data(comp_stats, filepath=None):
    fig, ax = plt.subplots(figsize=(6, 4))
    y = comp_stats['平均排名']
    x = comp_stats['出场次数']
    names = comp_stats.index
    ax.scatter(x, y)
    ax.set_title('阵容数据')
    ax.set_ylabel('平均排名')
    ax.set_xlabel('出场次数')
    ax.set_xlim(left=0, right=max(x) + 1)
    ax.set_ylim(bottom=1, top=8)
    ax.hlines(y=4.5, xmin=0, xmax=max(x) + 1, colors='orange', linestyles='--', lw=2, label='理论均值')
    ax.legend(loc="lower right")
    for i, txt in enumerate(names):
        ax.annotate(txt, (x[i] + 0.25, y[i]))
    fig.gca().invert_yaxis()
    if filepath:
        fig.savefig(filepath, dpi=250)
    else:
        plt.show(fig)
    plt.close(fig)

comp_stats = get_comp_statistics(df)
plot_comp_data(comp_stats, "data/comp_data.png")
# plot_comp_data(comp_stats, None)
ban_stats = get_ban_statistics(df)
plot_ban_to_comp(ban_stats, "data/ban_to_comp.png")
# plot_ban_to_comp(ban_stats, None)

In [7]:
weibo_csv_path = 'weibo_crawler/weibo/王者模拟战职业大师赛/5464294919.csv'
weibo_df = pd.read_csv(weibo_csv_path)
weibo_df = weibo_df.loc[weibo_df['正文'].str.contains('决赛圈')]
display(weibo_df.head(1))
print(weibo_df.shape)



,id,bid,正文,头条文章url,原始图片url,视频url,位置,日期,工具,点赞数,评论数,转发数,话题,@用户
3,4558135123513004,JoomkeJTm,王者荣耀超话 KPL超话 #王者模拟战# #王者模拟战职业大师赛#·秋季赛·常规赛第四周...,NaN,https://wx4.sinaimg.cn/large/005XNCJ1gy1gjj3p5...,NaN,NaN,2020-10-09,搜狗高速浏览器,0,0,0,"王者模拟战,王者模拟战职业大师赛",NaN


(25, 14)
